## Homework 5
This notebook contains the solutions for the fifth homework assignment of the Machine Learning Zoomcamp 2025 course, which can be viewed on github with the following [link](https://github.com/DataTalksClub/machine-learning-zoomcamp/blob/master/cohorts/2025/05-evaluation/homework.md). The final solutions notebook is submitted for grading at the following [link](https://courses.datatalks.club/ml-zoomcamp-2025/homework/hw05).
### Setup
- Install `uv`: `pip install uv`
- May need to create a virtual environment in `uv`: `uv init`
- Use the following command in a terminal to launch the notebook from the `uv` virtual environment: `uv run --with jupyter jupyter lab`

In [15]:
# Add packages to the `uv` project with the --active flag so they're available in the notebook
!uv add --active "scikit-learn==1.6.1" fastapi uvicorn pandas requests

Resolved 30 packages in 303ms                                        
Prepared 4 packages in 310ms                                             
Installed 4 packages in 4ms3.4.4                            
 + certifi==2025.10.5
 + charset-normalizer==3.4.4
 + requests==2.32.5
 + urllib3==2.5.0


In [16]:
from fastapi import FastAPI
import pandas as pd
import pickle
import re
import requests
import sklearn
import uvicorn

In [4]:
BOLD = "\033[1m"

### Q1. `uv` version
1. Install uv
2. Use --version to find which version of `uv` was installed

What's the version of uv?

In [5]:
uv_version_cl = !uv --version
uv_version = uv_version_cl[0].split()[1]
print(f"The `uv` version = {BOLD}{uv_version}")

The `uv` version = 0.9.5


### Q2. Use `uv` to install Scikit-Learn
Use `uv` to install Scikit-Learn version 1.6.1.

What's the first hash for Scikit-Learn you get in the lock file?
* Include the entire string starting with sha256
* Don't include quotes

In terminal, create uv project: `uv init`. Then add required version of scikit-learn: `uv add "scikit-learn==1.6.1"`

In [6]:
fname = "uv.lock"
is_package = False
is_skl = False
with open(fname, 'r') as f:
    for line in f:
        if line.startswith("[[package]]"):
            is_package = True
        if line.startswith('name = "scikit-learn"'):
            is_skl = True
        if is_package and is_skl and "hash" in line:
            hash_line = line
            break

In [7]:
match = re.search(r"hash = (.*?)(?:,|$)", hash_line)
hash_str = match.group(1).strip().replace('"', '')
print(f"The first hash value for the scikit-learn 1.6.1 package = {BOLD}{hash_str}")

The first hash value for the scikit-learn 1.6.1 package = sha256:b4fc2525eca2c69a59260f583c56a7557c6ccdf8deafdba6e060f94c1c59738e


### 3. Load model and score record
1. Write a script for loading the pipeline with pickle
2. Score this record: {
    "lead_source": "paid_ads",
    "number_of_courses_viewed": 2,
    "annual_income": 79276.0
}

What's the probability that this lead will convert?

In [8]:
# Load model with the following command in terminal (only need to run once):
# !wget https://github.com/DataTalksClub/machine-learning-zoomcamp/raw/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin

--2025-10-28 23:07:51--  https://github.com/DataTalksClub/machine-learning-zoomcamp/raw/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin [following]
--2025-10-28 23:07:52--  https://raw.githubusercontent.com/DataTalksClub/machine-learning-zoomcamp/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1300 (1.3K) [application/octet-stream]
Saving to: ‘pipeline_v1.bin’

pipeline_v1.bin

In [9]:
client_1 = {
    "lead_source": "paid_ads",
    "number_of_courses_viewed": 2,
    "annual_income": 79276.0
}

In [10]:
model_file = "pipeline_v1.bin"
with open(model_file, 'rb') as f_in:
    pipeline = pickle.load(f_in)

churn = pipeline.predict_proba(customer)[0, 1]

In [11]:
print(f"The probability that the customer will churn = {BOLD}{round(churn, 3)}")

The probability that the customer will churn = 0.534


### 4. Load model and score record
Serve this model as a web service
1. Install FastAPI
2. Write FastAPI code for serving the model
3. Score this client using requests:
```
url = "YOUR_URL"
client = {
    "lead_source": "organic_search",
    "number_of_courses_viewed": 4,
    "annual_income": 80304.0
}
requests.post(url, json=client).json()
```
What's the probability that this client will get a subscription?

In [12]:
client_2 = {
    "lead_source": "organic_search",
    "number_of_courses_viewed": 4,
    "annual_income": 80304.0
}

In [13]:
# Identify unique values for lead_score column for validation in pydantic class
url_path = "https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv"
cls_df = pd.read_csv(url_path)
cls_df['lead_source'].fillna("NA").unique()

array(['paid_ads', 'social_media', 'events', 'referral', 'organic_search',
       'NA'], dtype=object)

In [21]:
# Run the prediction service in a terminal (only needed once per session)
# `uvicorn predict:app --host 0.0.0.0 --port 9696 --reload`
# Then run the client script to get the converted probability
response = !python3 client.py

In [25]:
print(f"The probability that client 2 will get a subscription = {BOLD}{round(float(response[1]), 3)}")

The probability that client 2 will get a subscription = 0.534


### 5. Load docker image
Download the base image agrigorev/zoomcamp-model:2025. 
1. Make it by using `docker pull` command.
2. Use the "SIZE" column from `docker images`
3. Need sudo privilages
What's the size of this base image?

In [4]:
image_size = "121MB"
print(f"The agrigorev/zoomcamp-model:2025 model has a size of {image_size}")

The agrigorev/zoomcamp-model:2025 model has a size of 121MB
